In [1]:
import pandas as pd
import numpy as np
import os 
import os.path as osp
import glob
from datetime import datetime

In [2]:
import utils

In [3]:
os.getcwd()

'D:\\Project- Electro\\social_behavior_electro\\data_organization\\python'

In [4]:
main_folder_path = osp.join('..','..','..','','')
chamber_folder = osp.join(main_folder_path, 'summary - 3-2022','Brain states of Affiliation and Aversive behaviour')
chamber_affiliative_folder = osp.join(chamber_folder,'Affiliative interaction')
chamber_aversive_folder = osp.join(chamber_folder,'Aversive Interaction')
free_folder = osp.join(main_folder_path, 'Summary of free interaction - 1-2023')

all_folders = {
    'chamber_aff':chamber_affiliative_folder,
    'chamber_avv':chamber_aversive_folder,
    'free':free_folder
    }


### Folder formation:
* audio excel files - extracted vocalizations data excel files
* audioclips - matlab audioclip objects
* audioclip data files - matlab struct files with more data
* Audio-wav files - raw wav files
* LFP - filtered lowpass raw lfp data matlab files
* Multispikes - spike times matlab files
* SNIFF DATA - raw sniff recording data matlab files
* Timestemps - time stamps matlab files
* video files - extracted video data matlab files
* cropped_raw_video - cropped raw video avi files of the interactions only (only for free)
* video_pose_estimation - csv files with the pose extimation of the cropped files (only for free)

### Session parameters:
#### Identification
* rat_number
* day_number
* probe_number

#### Charactaristics
* paradigm - free, chamber
* sociability - affiliative, aversive (only in chamber sessions)


In [5]:
sub_folder_names = [
    'audio excel files',
    'audioclips',
    'audioclip data files',
    'Audio-wav files',
    'LFP',
    'Multispikes',
    'SNIFF DATA',
    'Timestemps',
    'video files',
    'cropped_raw_video',
    'video_pose_estimation'
]

session_parameters = [
    'rat_number',
    'day_number',
    'probe_number',
    'paradigm',
    'sociability'   
]

In [6]:
# f = 'free_Rat5-probe8-day1-Threshold_-3.5_MultiUnit_Spikes_Data'


In [10]:
# Go over all file names and extract all available sessions
all_file_details = []
#
for folder_type in all_folders.values():
    for sub_type in sub_folder_names:
        all_files_sub = glob.glob(osp.join(folder_type,sub_type,'*.[!ini]*'))
        all_files_names_sub = ['\\'.join(osp.split(a)[-2:]) for a in all_files_sub]
        for fname in all_files_names_sub:
            ret = utils.extract_details_from_file_names(fname)
            
            ret['filetype'] = sub_type
            ret['fullpath'] = osp.abspath(fname)            
#             ret['filepath'] = osp.join(*osp.split(osp.abspath(fname))[:-1])
#             ret['filename'] = osp.split(osp.abspath(fname))[-1]
            
#             if fname == f:
#                 print(ret)
            all_file_details.append(ret)
    


In [11]:
df = pd.DataFrame.from_dict(all_file_details)

In [12]:
# Go over all file names and extract all available sessions
all_session_names = []
all_file_names = []

for folder_type in all_folders.values():
    for sub_type in sub_folder_names:
        all_files_sub = glob.glob(osp.join(folder_type,sub_type,'*.[!ini]*'))
        all_files_names_sub = [''.join(osp.split(a)[-2:]) for a in all_files_sub]

        ret = [utils.extract_details_from_file_names(a) for a in all_files_names_sub]
        ret_v = [tuple(a.values()) for a in ret]

        all_file_names += all_files_names_sub
        all_session_names +=ret_v
    
all_session_ids = set(all_session_names)

In [13]:
df_resorted = df.set_index(session_parameters+['filetype']).unstack('filetype')
df_resorted.columns = df_resorted.columns.droplevel(0)
df_resorted.reset_index(inplace=True)
current_time_str = datetime.now().strftime("%Y%m%d-%H%M%S")
df_resorted.to_excel(f'file_to_session_mapping_{current_time_str}.xlsx')

In [ ]:
df_resorted

In [ ]:
df_resorted.columns